In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyts.image import MarkovTransitionField, GramianAngularField
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tqdm import tqdm
import seaborn as sns

In [2]:
from numba import njit

In [3]:
data = pd.read_csv('candles.csv').drop('Unnamed: 0', axis=1)

FileNotFoundError: [Errno 2] No such file or directory: 'candles.csv'

In [ ]:
data = np.array(data)

In [ ]:
def data_gen(n, data):
    
    total_prices = np.zeros((1, n, 4))
    for i in tqdm(range(0, len(data), n)):
        d_open = data[i:i+n, 0]
        d_high = data[i:i+n, 1]
        d_low = data[i:i+n, 2]
        d_close = data[i:i+n, 3]
        if i + n <= len(data):
            total_prices = np.concatenate((total_prices, np.concatenate([d_open.reshape(1, n), d_high.reshape(1, n), d_low.reshape(1, n), d_close.reshape(1, n)], axis=1).reshape(1, -1, 4)), axis=0)
    total_prices = total_prices[1:]
    np.random.shuffle(total_prices)
    return total_prices

In [ ]:
d = data_gen(100000, data)

In [ ]:
total_prices = data_gen(100, data)

In [ ]:
gasf = GramianAngularField(image_size=100, method='difference', sample_range=(0,1), overlapping=True)
total_prices1 = gasf.fit_transform(total_prices).reshape(-1, 100, 100, 1)
gasf = GramianAngularField(image_size=100, method='summation', sample_range=(0,1), overlapping=True)
total_prices2 = gasf.fit_transform(total_prices).reshape(-1, 100, 100, 1)
X = np.concatenate([total_prices1, total_prices2], axis=3)

In [ ]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import tensorflow
tensorflow.compat.v1.disable_eager_execution()

In [ ]:
np.random.seed(0)
nums = np.arange(len(X))
np.random.shuffle(nums)
X = X[nums]
total_prices = total_prices[nums]

In [ ]:
train_data = X[:6000]
test_data = X[6000:]

In [ ]:
import tensorflow

input_data = tensorflow.keras.layers.Input(shape=(100, 100, 2))

encoder = tensorflow.keras.layers.Conv2D(filters=512, kernel_size=5, activation='relu')(input_data)
encoder = tensorflow.keras.layers.MaxPooling2D(2)(encoder)
encoder = tensorflow.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu')(encoder)
encoder = tensorflow.keras.layers.MaxPooling2D(2)(encoder)
encoder = tensorflow.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu')(encoder)
encoder = tensorflow.keras.layers.MaxPooling2D(2)(encoder)
encoder = tensorflow.keras.layers.Flatten()(encoder)
encoder = tensorflow.keras.layers.Dense(64)(encoder)

In [ ]:
def sample_latent_features(distribution):
    distribution_mean, distribution_variance = distribution
    batch_size = tensorflow.shape(distribution_variance)[0]
    random = tensorflow.keras.backend.random_normal(shape=(batch_size, tensorflow.shape(distribution_variance)[1]))
    return distribution_mean + tensorflow.exp(0.5 * distribution_variance) * random

In [ ]:
distribution_mean = tensorflow.keras.layers.Dense(64, name='mean')(encoder)
distribution_variance = tensorflow.keras.layers.Dense(64, name='log_variance')(encoder)
latent_encoding = tensorflow.keras.layers.Lambda(sample_latent_features)([distribution_mean, distribution_variance])

In [ ]:
encoder_model = tensorflow.keras.Model(input_data, latent_encoding)
encoder_model.summary()

In [ ]:
decoder_input = tensorflow.keras.layers.Input(shape=(64))
decoder = tensorflow.keras.layers.Dense(10*10*128, activation='relu')(decoder_input)
decoder = tensorflow.keras.layers.Reshape((10, 10, 128))(decoder)
decoder = tensorflow.keras.layers.UpSampling2D(2)(decoder)
decoder = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, activation='relu')(decoder)
decoder = tensorflow.keras.layers.UpSampling2D(2)(decoder)
decoder = tensorflow.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, activation='relu')(decoder)
decoder = tensorflow.keras.layers.UpSampling2D(2)(decoder)
decoder = tensorflow.keras.layers.Conv2DTranspose(filters=512, kernel_size=3, activation='relu')(decoder)
decoder_output = tensorflow.keras.layers.Conv2DTranspose(filters=2, kernel_size=7, activation='sigmoid')(decoder)

In [ ]:
decoder_model = tensorflow.keras.Model(decoder_input, decoder_output)
decoder_model.summary()

In [ ]:
encoded = encoder_model(input_data)
decoded = decoder_model(encoded)

In [ ]:
autoencoder = tensorflow.keras.models.Model(input_data, decoded)

In [ ]:
def get_loss(distribution_mean, distribution_variance):
    
    def get_reconstruction_loss(y_true, y_pred):
        reconstruction_loss = tensorflow.keras.losses.mse(y_true, y_pred)
        reconstruction_loss_batch = tensorflow.reduce_mean(reconstruction_loss)
        return reconstruction_loss_batch*100*1
    
    def get_kl_loss(distribution_mean, distribution_variance):
        kl_loss = 1 + distribution_variance - tensorflow.square(distribution_mean) - tensorflow.exp(distribution_variance)
        kl_loss_batch = tensorflow.reduce_mean(kl_loss)
        return kl_loss_batch*(-0.5)
    
    def total_loss(y_true, y_pred):
        reconstruction_loss_batch = get_reconstruction_loss(y_true, y_pred)
        kl_loss_batch = get_kl_loss(distribution_mean, distribution_variance)
        return reconstruction_loss_batch + kl_loss_batch
    
    return total_loss

In [ ]:
autoencoder.compile(loss=get_loss(distribution_mean, distribution_variance), optimizer=tensorflow.keras.optimizers.Adam(lr=0.0001))
autoencoder.summary()

In [ ]:
autoencoder.fit(train_data, train_data, epochs=50, batch_size=64, validation_data=(test_data, test_data), callbacks=[ReduceLROnPlateau(patience=20, min_lr=0.00001), EarlyStopping(patience=40)])

In [ ]:
autoencoder.save('autoencoder_RAW.hdf5')

In [ ]:
encoder_model.save('encoder.hdf5')
decoder_model.save('decoder.hdf5')

In [ ]:
# autoencoder.load_weights('autoencoder_GADSF.hdf5')

In [ ]:
class ClusteringLayer(tensorflow.keras.layers.Layer):
    '''
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.
    '''

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = tensorflow.keras.layers.InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = tensorflow.keras.layers.InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(name='clusters', shape=(self.n_clusters, input_dim), initializer='glorot_uniform') 
        
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        ''' 
        student t-distribution, as used in t-SNE algorithm.
        It measures the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
       
        inputs: the variable containing data, shape=(n_samples, n_features)
        
        Return: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        '''
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure all of the values of each sample sum up to 1.
        
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
clustering_layer = ClusteringLayer(2, name='clustering')(encoder_model.output)
model = Model(inputs=encoder_model.input, outputs=clustering_layer)

In [ ]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(0.0001), loss='kld')

In [ ]:
kmeans = KMeans(n_clusters=2, n_init=20)
y_pred = kmeans.fit_predict(encoder_model.predict(train_data))

In [ ]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

In [ ]:
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [ ]:
loss = 0
index = 0
maxiter = 1000
update_interval = 100
index_array = np.arange(train_data.shape[0])

In [ ]:
tol = 0.001

In [ ]:
for ite in tqdm(range(int(maxiter))):
    if ite % update_interval == 0:
        q = model.predict(train_data, verbose=0)
        p = target_distribution(q)

    idx = index_array[index * 32: min((index+1) * 32, train_data.shape[0])]
    loss = model.train_on_batch(x=train_data[idx], y=p[idx])
    index = index + 1 if (index + 1) * 32 <= train_data.shape[0] else 0


In [ ]:
q = model.predict(test_data, verbose=0)
p = target_distribution(q)
y_pred = q.argmax(1)


In [ ]:
total_prices[6000:].shape

In [ ]:
total_prices[6000:]

In [ ]:
# pd.DataFrame(total_prices[6000:]).to_csv('data.csv')

In [ ]:
np.save('label_MTF.npy',y_pred)

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score
s_scores = []
d_scores = []
for i in range(2, 9):
    autoencoder.load_weights('autoencoder_RAW.hdf5')
    clustering_layer = ClusteringLayer(i, name='clustering')(encoder_model.output)
    model = Model(inputs=encoder_model.input, outputs=clustering_layer)
    model.compile(optimizer=tensorflow.keras.optimizers.Adam(0.0001), loss='kld')
    kmeans = KMeans(n_clusters=i, n_init=20)
    y_pred = kmeans.fit_predict(encoder_model.predict(train_data))
    model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
    loss = 0
    index = 0
    maxiter = 1000
    update_interval = 100
    index_array = np.arange(train_data.shape[0])
    tol = 0.001
    for ite in tqdm(range(int(maxiter))):
        if ite % update_interval == 0:
            q = model.predict(train_data, verbose=0)
            p = target_distribution(q)

        idx = index_array[index * 32: min((index+1) * 32, train_data.shape[0])]
        loss = model.train_on_batch(x=train_data[idx], y=p[idx])
        index = index + 1 if (index + 1) * 32 <= train_data.shape[0] else 0
    latent = encoder_model.predict(test_data)
    model_output = model.predict(test_data)
    labels = []
    for row in model_output:
        labels.append(np.argmax(row))
    s_scores.append(silhouette_score(latent,labels))
    d_scores.append(davies_bouldin_score(latent, labels))

In [ ]:
s_scores

In [ ]:
d_scores

In [ ]:
import pickle

In [ ]:
with open("s_scores_RAW.txt", "wb") as fp:
    GADSF_s_scores = pickle.dump(s_scores, fp)
with open("d_scores_RAW.txt", "wb") as fp:
    GADSF_d_scores = pickle.dump(d_scores, fp)

In [ ]:
with open("s_scores_GADSF.txt", "rb") as fp:
    GADSF_s_scores = pickle.load(fp)
with open("d_scores_GADSF.txt", "rb") as fp:
    GADSF_d_scores = pickle.load(fp)
with open("s_scores_GASF.txt", "rb") as fp:
    GASF_s_scores = pickle.load(fp)
with open("d_scores_GASF.txt", "rb") as fp:
    GASF_d_scores = pickle.load(fp)
with open("s_scores_GADF.txt", "rb") as fp:
    GADF_s_scores = pickle.load(fp)
with open("d_scores_GADF.txt", "rb") as fp:
    GADF_d_scores = pickle.load(fp)
with open("s_scores_MTF.txt", "rb") as fp:
    MTF_s_scores = pickle.load(fp)
with open("d_scores_MTF.txt", "rb") as fp:
    MTF_d_scores = pickle.load(fp)
with open("s_scores_RAW.txt", "rb") as fp:
    RAW_s_scores = pickle.load(fp)
with open("d_scores_RAW.txt", "rb") as fp:
    RAW_d_scores = pickle.load(fp)

In [ ]:
Sil = np.array([GADSF_s_scores, GASF_s_scores, GADF_s_scores, MTF_s_scores, RAW_s_scores])
indexes = ['GASF + GADF', 'GASF', 'GADF', 'MTF', 'Raw data']
dSil = pd.DataFrame(Sil, columns=['2_clusters', '3_clusters','4_clusters','5_clusters','6_clusters','7_clusters','8_clusters'])
dSil.index = indexes

In [ ]:
dSil.to_excel('Sil.xls')

In [ ]:
Dav = np.array([GADSF_d_scores, GASF_d_scores, GADF_d_scores, MTF_d_scores, RAW_d_scores])
indexes = ['GASF + GADF', 'GASF', 'GADF', 'MTF', 'Raw data']
dSil = pd.DataFrame(Dav, columns=['2_clusters', '3_clusters','4_clusters','5_clusters','6_clusters','7_clusters','8_clusters'])
dSil.index = indexes

In [ ]:
dSil.to_excel('Dav.xls')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
plt.title('Silhouette', fontsize=30)
Xs = [2, 3, 4, 5, 6, 7, 8]
plt.plot(Xs, GADSF_s_scores, c="blue", label="GADSF")
plt.plot(Xs, GASF_s_scores, c="red", label="GASF")
plt.plot(Xs, GADF_s_scores, c="green", label="GADF")
plt.plot(Xs, MTF_s_scores, c="yellow", label="MTF")
plt.plot(Xs, RAW_s_scores, c="purple", label="RAW")
plt.tick_params(labelsize=20)
plt.xlabel('Number of clusters', fontsize=30)
plt.ylabel('Silhouette score', fontsize=30)
plt.legend(loc="upper right", fontsize=30)
plt.savefig('Silhouette.png')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
plt.title('Davies Bouldin', fontsize=30)
Xs = [2, 3, 4, 5, 6, 7, 8]
plt.plot(Xs, GADSF_d_scores, c="blue", label="GADSF")
plt.plot(Xs, GASF_d_scores, c="red", label="GASF")
plt.plot(Xs, GADF_d_scores, c="green", label="GADF")
plt.plot(Xs, MTF_d_scores, c="yellow", label="MTF")
plt.plot(Xs, RAW_d_scores, c="purple", label="RAW")
plt.tick_params(labelsize=20)
plt.xlabel('Number of clusters', fontsize=30)
plt.ylabel('Davies Bouldin score', fontsize=30)
plt.legend(loc="upper right", fontsize=30)
plt.savefig('Davies_Bouldin.png')

In [ ]:
s_scores

In [ ]:
d_scores

In [ ]:
with open("s_scores_MTF.txt", "wb") as fp:
    pickle.dump(s_scores, fp)
with open("d_scores_MTF.txt", "wb") as fp:
    pickle.dump(d_scores, fp)

In [ ]:
model.load_weights('model_.hdf5')

In [ ]:
#model.load_weights('model.hdf5')

In [ ]:
q = model.predict(test_data, verbose=0)
p = target_distribution(q)

In [ ]:
y_pred = q.argmax(1)

In [ ]:
np.unique(y_pred[:50])

In [ ]:
y_pred[:50]

In [ ]:
plt.plot(total_prices[6000:][0])

In [ ]:
plt.plot(total_prices[6000:][2])

In [ ]:
plt.plot(total_prices[6000:][4])

In [ ]:
f = plt.figure(figsize=(15,3))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax.plot(total_prices[6000:][0])
ax.set_title('GASF+GADF - Cluster 0')
ax2.plot(total_prices[6000:][2])
ax2.set_title('GASF+GADF - Cluster 1')
plt.show()
f.savefig('gasdf.jpg')

In [ ]:
ls = [f'Cluster {i}' for i in labels]

In [ ]:
sns.scatterplot(x=X_embedded[:,0], y=X_embedded[:,1], hue=ls, )

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [ ]:
latent = encoder_model.predict(test_data)
model_output = model.predict(test_data)
labels = []

In [ ]:
for row in model_output:
    labels.append(np.argmax(row))

In [ ]:
silhouette_score(latent,labels)

In [ ]:
# davies_bouldin_score(latent,labels)

In [ ]:
# for row in model_output:
#     labels.append(np.argmax(row))

In [ ]:
# labels

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(latent)
X_embedded.shape

In [ ]:
ls = []
cs = ['red', 'green', 'blue']
for label in labels:
    ls.append(cs[label])

In [ ]:
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=ls)

In [ ]:
plt.plot(total_prices[6000:][0])

In [ ]:
f = plt.figure(figsize=(15,3))
ax = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)
ax.plot(total_prices[6000:][0])
ax2.plot(total_prices[6000:][1])
ax3.plot(total_prices[6000:][5])
#